# Service Execution
This notebooks showcases a demo of the [APEx Dispatch API](https://github.com/ESA-APEx/apex_dispatch_api) for executing a service. In this notebook we will perform the execution for one of the services in the [APEx Algoritm Services Catalogue](https://algorithm-catalogue.apex.esa.int/), specfically the [PV Farm Detection](https://algorithm-catalogue.apex.esa.int/apps/eurac_pv_farm_detection#description).

In [1]:
%pip install esa-apex-algorithms


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import rasterio
import numpy as np
import tempfile
import asyncio
import json
import websockets
import httpx
import io
import base64
import time
from ipyleaflet import ImageOverlay
from PIL import Image
from ipyleaflet import Map, GeoJSON, TileLayer
from rasterio.warp import transform_bounds
from shapely.geometry import shape
from pyproj import Transformer
from authlib.integrations.requests_client import OAuth2Session
from urllib.parse import urlparse, parse_qs
from esa_apex_toolbox.algorithms import GithubAlgorithmRepository

In [3]:
def bbox_to_geojson(bbox: dict) -> dict:
    west, south, east, north = bbox["west"], bbox["south"], bbox["east"], bbox["north"]
    return {
        "type": "Polygon",
        "coordinates": [
            [
                [west, south],
                [east, south],
                [east, north],
                [west, north],
                [west, south]
            ]
        ]
    }

In [4]:
def geojson_to_bbox(polygon):
    coords = polygon["coordinates"][0]
    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]
    west, south, east, north = min(lons), min(lats), max(lons), max(lats)
    
    # Convert back to original bounding box dict
    return {
        "west": west,
        "south": south,
        "east": east,
        "north": north
    }


## Look up the algorithm to execute

In [5]:
repo = GithubAlgorithmRepository(
            owner="ESA-APEx",
            repo="apex_algorithms",
            folder="algorithm_catalog",
        )

In [6]:
repo.list_algorithms()

['wind_turbine',
 'eurac_pv_farm_detection',
 'gep_bas',
 'gep_ost',
 'sar_coin',
 'snap_insar_sentinel1_iw_slc',
 'bap_composite',
 'biopar',
 'fusets_mogpr',
 'max_ndvi',
 'max_ndvi_composite',
 'parcel_delineation',
 'random_forest_firemapping',
 'sentinel1_stats',
 'variabilitymap',
 'worldcereal_crop_extent',
 'worldcereal_crop_type',
 'worldcover_statistics',
 'worldagrocommodities']

In [7]:
service = repo.get_algorithm('eurac_pv_farm_detection')

In [8]:
service

Algorithm(id='eurac_pv_farm_detection', title='Photovoltaic farms mapping', description='Demonstrator service for the detection of photovoltaic farms. Photovoltaic farms (PV farms) mapping is essential for establishing valid policies regarding natural resources management and clean energy. ', udp_link=UdpLink(href='https://raw.githubusercontent.com/ESA-APEx/apex_algorithms/refs/heads/main/algorithm_catalog/eurac/eurac_pv_farm_detection/openeo_udp/eurac_pv_farm_detection.json', title='openEO Process Definition'), service_links=[ServiceLink(href='https://openeofed.dataspace.copernicus.eu', title='CDSE openEO federation')], license=None, organization='Eurac Research')

## Definition of parameters

In [9]:
dispatch_api = "dispatch-api.dev.apex.esa.int"

In [10]:
spatial_extent =  {
  "type": "Polygon",
  "coordinates": [
    [
      [16.342, 47.962],
      [16.414, 47.962],
      [16.414, 48.008],
      [16.342, 48.008],
      [16.342, 47.962]
    ]
  ]
}
temporal_extent = ["2023-05-01", "2023-09-30"]
output_format = "gtiff"

In [11]:
# Map related settings
center = shape(spatial_extent).centroid
zoom = 12

In [12]:
# Create a map centered at the approximate center of the area of interest
m = Map(center=[center.y, center.x], zoom=zoom)
 
# Add the tiles (GeometryCollection) to the map
geo_json = GeoJSON(data=spatial_extent)
m.add_layer(geo_json)

# Display the map
m

Map(center=[47.985, 16.378000000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…

## Authentication with the API
To access the different endpoints of the Dispatcher API it is important to first authenticate yourself with the APEx environment.

In [13]:
KEYCLOAK_HOST = "auth.dev.apex.esa.int"
CLIENT_ID = "apex-dispatcher-api-dev"

In [14]:
# Endpoints
authorization_endpoint = f"https://{KEYCLOAK_HOST}/realms/apex/protocol/openid-connect/auth"
token_endpoint = f"https://{KEYCLOAK_HOST}/realms/apex/protocol/openid-connect/token"

# Global token store
_token_data = None

def get_access_token():
    """
    Returns a valid access token. Refreshes it automatically if expired.
    """
    global _token_data

    # If we have a token and it hasn't expired yet, return it
    if _token_data and _token_data.get("expires_at", 0) > time.time() + 10:
        return _token_data["access_token"]

    # If token exists but is expired and has a refresh_token, refresh it
    if _token_data and "refresh_token" in _token_data:
        session = OAuth2Session(CLIENT_ID, token=_token_data)
        _token_data = session.refresh_token(token_endpoint)
        return _token_data["access_token"]

    # Otherwise, start a new OAuth2 flow
    session = OAuth2Session(
        client_id=CLIENT_ID,
        redirect_uri="http://localhost:8000/callback"
    )
    uri, state = session.create_authorization_url(authorization_endpoint)
    print("Open this URL in your browser:", uri)
    redirect_url = input("Paste the redirect URL here: ")
    parsed = urlparse(redirect_url)
    code = parse_qs(parsed.query).get("code")[0]

    _token_data = session.fetch_token(
        token_endpoint,
        code=code,
        client_secret=None,  # only if your client is confidential
        include_client_id=True
    )

    return _token_data["access_token"]

## Launching the service execution task

Next we trigger the service execution task on the dispatcher. We provide the details of the processing job that needs to be executed. The result is the information on the created service execution task.

In [18]:
execution_task = requests.post(
    f"http://{dispatch_api}/unit_jobs", 
    headers={
        "Authorization": f"Bearer {get_access_token()}"        
    },
    json={
        "title": "PV Farm Detection",
        "label": "openeo",
        "service": {
            "endpoint": service.service_links[0].href,
            "application": service.udp_link.href
        },
        "format": output_format,
        "parameters": {
            "spatial_extent": geojson_to_bbox(spatial_extent),
            "temporal_extent": temporal_extent
        }
    }
)
print(execution_task.json())
execution_task_id = execution_task.json()['id']

{'detail': 'An error occurred while creating the processing job: No OpenEO backend configuration found for URL: https://openeofed.dataspace.copernicus.eu'}


KeyError: 'id'

## Retrieve status of the service execution task
We can now write a continuous monitoring process that fetches the status of the service execution task and showcase the results on the map.

In [16]:
def add_cog_layer(cog_url, name=None, m=m):
    with rasterio.open(cog_url) as src:
        band = src.read(1).astype(np.float32)

        bounds = transform_bounds(src.crs, "EPSG:4326", *src.bounds)

        # Normalize 0–255
        band = 255 * (band - band.min()) / (band.max() - band.min())
        band = band.astype(np.uint8)

    # Convert to PNG data URI
    buf = io.BytesIO()
    Image.fromarray(band).save(buf, format="PNG")
    data_url = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode("utf-8")

    bbox = ((bounds[1], bounds[0]), (bounds[3], bounds[2]))
    overlay = ImageOverlay(url=data_url, bounds=bbox, name=name or "Gray COG")
    m.add_layer(overlay)
    return overlay

def add_geojson_layer(url,name=None, m=m):
    data = requests.get(url).json()
    transformer = Transformer.from_crs(data["crs"]["properties"]["name"], "EPSG:4326", always_xy=True)

    for feature in data["features"]:
        geom = feature["geometry"]
        if geom["type"] == "Polygon":
            new_coords = []
            for ring in geom["coordinates"]:
                new_ring = [transformer.transform(x, y) for x, y in ring]
                new_coords.append(new_ring)
            geom["coordinates"] = new_coords
    geo_json = GeoJSON(data=data)
    m.add_layer(geo_json)

In [17]:
# Function to style jobs
def job_style(feature):
    status = feature["properties"]["status"]
    color = {
        "created": "blue",
        "queued": "orange",
        "running": "yellow",
        "finished": "green",
        "canceled": "gray",
        "failed": "red"
    }.get(feature["properties"]["status"], "black")
    return {
        "color": color,
        "fillColor": color,
        "fillOpacity": 0.5 if status != "finished" else 0.0
    }


m = Map(center=[center.y, center.x], zoom=zoom)
geo_json = GeoJSON(
    data={
        "type": "FeatureCollection",
        "features": []
    }
)
geo_json.style_callback = job_style
m.add_layer(geo_json)
m.layout.height = '1000px'
display(m)

async def show_results(job_id):
    async with httpx.AsyncClient() as client:
        result = await client.get(f"http://{dispatch_api}/unit_jobs/{job_id}/results", headers={
            "Authorization": f"Bearer {get_access_token()}"
        })
        response = result.json()
        if output_format.lower() == "geojson":
            result = response["assets"]["vectorcube.geojson"]["href"]
            add_geojson_layer(result, name=f"Job {job_id}", m=m)
        else:
            cog = response["assets"]["openEO.tif"]["href"]
            add_cog_layer(cog, name=f"Job {job_id}", m=m)
        return response

async def listen_for_updates():
    finished = False
    while not finished:
        response  =requests.get(
            f"http://{dispatch_api}/unit_jobs/{execution_task_id}", 
            headers={
                "Authorization": f"Bearer {get_access_token()}"        
            })
        job = response.json()        
        geo_json.data = {
            "type": "FeatureCollection",
            "features": [{
                "type": "Feature",
                "geometry": bbox_to_geojson(job["parameters"]["spatial_extent"]),
                "properties": {
                    "status": job["status"],
                }
            }]
        }
        
        if job["status"] == "finished":
            await show_results(job["id"])

        finished = job['status'] in ["finished", "canceled", "failed"]
        time.sleep(3)
        

# Run the websocket listener in the notebook
await listen_for_updates()

Map(center=[47.985, 16.378000000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…